패키지 로드

In [ ]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

인플루언서 리스트

In [ ]:
influencer_list = [
                    'sea__color', 'h._cherish', 'howaremee', 'wangsubin', 'hyejxong__', 'jxnjuvv',
                    '__ryeong', 'mayay____y', '_hwoniii', 'chaevely___', 'seon_uoo', 'na_onion',
                    'soena_td', 'b_in__b', 'iam.fleur', 'jesica0908', '81bbuni', 'jaymee4u',
                    '__kindacool', 'mona_monanim', 'jxxsng', 'xx.s_on', 'c.ssung', 'holiday__365',
                    'yo_ho__12', 'mmars_20', 'd.b_choi', 'i_m_fe', 'foured', 'h___ryan', 'leojmakeup', 
                    'c_eunheee', 'chiyoon', 'haha_camping', 'sh_nn', 'ggarinam', '_ralphu_',  'h_ryan', 
                    'oddyune', 'bolm2', 'sun_nyee', 'songyibom', 'nugayoung', 'hwan_s._', 'sacompanysun'
                    ]

인스타그램 로그인 후 접속

In [ ]:
# 로그인 아이디/비밀번호 입력
strId = 
strPassword =

In [ ]:
# 인스타 로그인
# 크롬 드라이버 경로 설정
path = r"C:/Users/NT550-048/Desktop/chromedriver.exe"
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.get('https://www.instagram.com/')
time.sleep(2)
id_input = driver.find_elements(By.XPATH, '//*[@id="loginForm"]/div/div[1]/div/label/input')[0]
id_input.send_keys(strId)
pw_input = driver.find_elements(By.XPATH, '//*[@id="loginForm"]/div/div[2]/div/label/input')[0]
pw_input.send_keys(strPassword)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]').click()
time.sleep(3)

크롤링할 인플루언서 리스트로 변환

In [ ]:
# 크롤링할 계정 리스트
ids = ['i_m_fe', 'iam.fleur']

크롤링 수행

In [ ]:
# 인플루언서 여러명 크롤링
influ = []
for id in ids:
    driver.get(f'https://www.instagram.com/{id}') # 인플루언서 인스타 접속
    time.sleep(5)
    # 인플루언서 기본 정보 수집
    name = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/div[1]/a/h2').text # 계정 이름
    intro = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/div[3]').text # 소개
    content_num = int(driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/ul/li[1]/span').text) # 게시글 수
    followers = int(float(driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/ul/li[2]/a/span').text.split('만')[0])*10000) # 팔로워 수
    following = int(driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/section/main/div/header/section/ul/li[3]/a/span').text) # 팔로잉 수
    # 첫 게시물 클릭
    driver.find_element(By.CLASS_NAME, '_aagu').click()
    time.sleep(3)
    # 게시물 내용 (없는 경우 None)
    for n in range(1, content_num + 1):
        try:
            content_text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/h1').text
        except:
            content_text = 'None'
        # 게시글 시각
        content_time = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/div/div/a/span/time').get_attribute('datetime')
        content_time = datetime.strptime(content_time, '%Y-%m-%dT%H:%M:%S.000Z')
        # 게시물 좋아요 수 (여러명으로 표기시 -1)           
        try:
            content_like = re.findall(r'\d+', driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/span/a[2]/span').text)
        except:
            try:
                content_like = re.findall(r'\d+', driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/span/a/span').text)
            except:
                content_like = -1
        if content_like == []:
            content_like = -1
        elif content_like != -1:
            if len(content_like)==2:
                content_like = int(float(content_like[0] + '.' + content_like[1]) * 10000)
            else:
                content_like = int(content_like[0])
        # 게시물 사진 수 (0=릴스)
        try:
            num_photo = len(driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[1]/div/div[2]').find_elements(By.CLASS_NAME, '_acnb'))
        except:
            num_photo = 0
        # 댓글 더보기 없을 때까지 클릭
        btn = True
        while btn == True:
            try:
                driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button').click()
                time.sleep(1)
            except:
                btn = False
        # 댓글 정보 수집
        try:
            # 댓글 수
            reply_box = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul')
            reply_tot = len(reply_box.find_elements(By.CLASS_NAME, '_a9ym'))
            for num_reply in range(1, reply_tot + 1):
                # 댓글 내용
                reply = driver.find_element(By.XPATH, f'/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[{num_reply}]/div/li/div/div/div[2]/div[1]/span').text
                # 댓글 시각
                reply_time = driver.find_element(By.XPATH, f'/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[{num_reply}]/div/li/div/div/div[2]/div[2]/span/a/time').get_attribute('datetime')
                reply_time = datetime.strptime(reply_time, '%Y-%m-%dT%H:%M:%S.000Z')
                # 댓글 좋아요 수 (없으면 0)
                try:
                    reply_like = int(re.findall(r'\d+', driver.find_element(By.XPATH, f'/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[{num_reply}]/div/li/div/div/div[2]/div[2]/span/button[1]/span').text)[0])
                except:
                    reply_like = 0
                influ.append([name, intro, content_num, followers, following, content_text, content_time, num_photo, content_like, reply_tot, reply, reply_time, reply_like])
        except:pass
        time.sleep(3)
        # 다음 게시글 없을 때까지 클릭
        if n == 1:
            driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[1]/div/div/div/button').click()
        else:
            try:
                driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div[1]/div/div/div[2]/button').click()
                time.sleep(3)
            except:
                break
influ_df = pd.DataFrame(influ, columns=['name', 'intro', 'content_num', 'followers', 'following', 'content_text', 'content_time', 'num_photo', 'content_like', 'reply_tot', 'reply', 'reply_time', 'reply_like'])